In [7]:
import pandas as pd
import numpy as np

gun_homicide = pd.read_csv('./Datasets/Gun Homicide Information/Firearm_suicide_homicide_dataset.csv')
ai_incidents = pd.read_excel('./Datasets/AIAAIC Repository.xlsx', 1)
superstore_orders = pd.read_excel('./Datasets/Superstore.xlsx')

In [8]:
# Dataset de homicidios en Estados Unidos entre 1949 y 2020.
gun_homicide

,year,state,division,total_population,fss,homicide_rate,firearm_homicide_rate,nonfirearm_homicide_rate,firearm_suicides,total_suicides,firearm_homicides,nonfirearm_homicides,total_homicides,white_fss,nonwhite_fss,nextyearfss,nextyearnonwhitefss,nextyearwhitefss
0,1949,Alabama,East South Central,3004000.0,0.717949,14.28095872,7.822902796,6.458055925,140,195,235.0,194.0,429,0.708571,0.482655,0.688995,0.807692,0.672131
1,1949,Arizona,Mountain,726000.0,0.575000,5.64738292,3.168044077,2.479338843,46,80,23.0,18.0,41,0.583333,0.800000,0.589474,0.625000,0.586207
2,1949,Arkansas,West South Central,1835000.0,0.671053,7.901907357,4.141689373,3.760217984,102,152,76.0,69.0,145,0.664336,0.500000,0.644444,0.750000,0.632812
3,1949,California,Pacific,10499000.0,0.382623,4.61948757,1.857319745,2.762167826,709,1853,195.0,290.0,485,0.382187,0.777778,0.378732,0.367647,0.379142
4,1949,Colorado,Mountain,1249000.0,0.551570,5.044035228,3.442754203,1.601281025,123,223,43.0,20.0,63,0.559633,0.392405,0.462687,0.000000,0.467337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3646,2020,Virginia,South Atlantic,8590563.0,0.580350,6.181201395,5.121899461,1.059301934,697,1201,440.0,91.0,531,0.594540,0.514151,NaN,NaN,NaN
3647,2020,Washington,Pacific,7693612.0,0.509901,4.185290342,2.742534976,1.442755366,618,1212,211.0,111.0,322,0.531191,0.363636,NaN,NaN,NaN
3648,2020,West Virginia,South Atlantic,1784787.0,0.621469,6.387316806,4.874531247,1.512785559,220,354,87.0,27.0,114,0.626113,0.529412,NaN,NaN,NaN
3649,2020,Wisconsin,East North Central,5832655.0,0.510393,5.726380182,4.337647264,1.388732918,442,866,253.0,81.0,334,0.516169,0.435484,NaN,NaN,NaN


In [9]:
# Dataset que contiene informacion acerca de incidentes y controversias relacionados a la IA reportados en los medios.
ai_incidents

,AIAAIC Repository (beta),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,AIAAIC ID#,Title,Type,Released,Occurred,Country(s),Sector(s),Operator(s),Developer(s),System name(s),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AIAAIC1090,Google Autocomplete Texas massacre Antifa cons...,Incident,2008,2017,USA,Politics,Alphabet/Google/YouTube,Alphabet/Google/YouTube,Google Autocomplete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AIAAIC1089,Google Autocomplete,System,2008,NaN,Argentina; Australia; France; Germany; Hong Ko...,Banking/financial services; Business/professio...,Alphabet/Google/YouTube,Alphabet/Google/YouTube,Google Autocomplete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AIAAIC1088,"ChatGPT generates political messages, campaigns",Issue,NaN,2023,USA,Politics,OpenAI,OpenAI,ChatGPT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,AIAAIC005,Google Autocomplete conflates Bettina Wulff wi...,Incident,2008,2012,Germany,Politics,Alphabet/Google,Alphabet/Google,Google Autocomplete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1082,AIAAIC004,"Google Autocomplete says Rupert Murdoch, Jon H...",Incident,2008,2012,France,Media/entertainment/sports/arts,Alphabet/Google,Alphabet/Google,Google Autocomplete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083,AIAAIC003,Google Autocomplete falsely associates Japanes...,Incident,2008,2012,Japan,Private - individual,Alphabet/Google,Alphabet/Google,Google Autocomplete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1084,AIAAIC002,Google Images links music promoter to criminal...,Incident,2008,2012,Australia,Media/entertainment/sports/arts,Alphabet/Google,Alphabet/Google,Google Images; Google Autocomplete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Dataset que contiene ordenes de compra de una megatienda en estados unidos.
superstore_orders

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2013-138688,2013-06-13,2013-06-17,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2011-110422,2011-01-22,2011-01-24,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2014-121258,2014-02-27,2014-03-04,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2014-121258,2014-02-27,2014-03-04,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2014-121258,2014-02-27,2014-03-04,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200
